<p><img alt="banner" height="252px" width="1080px" src="https://docs.google.com/uc?export=download&id=1YJrz-tzQUkofEE37sRUdlCbnXf10gJlF"  align="center" hspace="10px" vspace="0px"></p>


# <font color='056938'> **Introduccción** </font>

La visualización de datos es una parte importante de los procesos de análisis en ciencia de datos. Estas visualizaciones sirven como herramientas exploratorias al inicio del ciclo del ánalisis para tener ideas preliminares y un mejor entendimiento de los datos. Tambien se usan visualizaciones para soportar hipotesis y soportar resultados. Finalmente, las visualizaciones también son la forma de comunicar los hallazgos y conclusiones del ánalisis al final del ciclo del análisis de datos.







## <font color='8EC044'> **Un meta para esta sesisón** </font>

Nuestro objetivo es que al finalizar esta sesión  estes en capacidad de:

> Visualizar un conjunto de datos de forma interactiva, haciendolos accesibles a través de funciones mediante el uso de librerias de python.



Usaremos como datos de estudio la información que el Departamento Administrativo Nacional de Estadística (`DANE`) recopila respecto a la producción agrícola en Colombia y su comercialización. Para ello accederemos a los datos que la entidad consolida en el Sistema de información de precios [SIPSA](https://www.dane.gov.co/index.php/servicios-al-ciudadano/servicios-informacion/sipsa).





# <font color='056938'> **Lectura de datos** </font>

En esta sesión no abordaremos a detalle los componentes para la adquisición y transformación de los datos. Sin embargo, es importante saber que esta tarea puede hacerse a través de la librería `pandas` que ya cubrimos en este curso. Esta es la descripción de la libreria que dan sus propios los creadores:

> **Pandas** es una herramienta de código abierto para la manipulación y análisis de datos que es rápida, poderosa, flexible y fácil de usar construida en el lenguaje de prgramación python.  

La libreria pandas permite leer facilmente archivos de datos en distintos formatos y almacenarlos en dataframes.

> Un **DataFrame** es una estructura de datos con dos dimensiones en la cual se puede guardar datos de distintos tipos (como caractéres, enteros, valores de punto flotante, factores y más) en columnas. Es similar a una hoja de cálculo o una tabla de SQL

Si lo considera necesario puede refrescar algunos de los conceptos de la libreria repasando el [notebook](https://drive.google.com/file/d/1xIOesdAYII1s-EM5hbB7UglvKofx44T4/view?usp=sharing) de la clase


## <font color='8EC044'> **Leer datos desde github** </font>

Dado que la lectura de datos puede tomar un tiempo considerable, hemos dispuesto una base de datos preprocesada que puede leerse directamente desde `github`



In [ ]:
import pandas as pd

!gdown 1YKS6g80aELebRd3SNwCwpYVyzedtCasD
!gdown 1w2awQieBE84Qoo1iA_di3CRKBcHF2AKm


# Leer datos preprocesados
df_precioProm = pd.read_csv('precioProm.csv', index_col=0)
df_promRec = pd.read_csv('recProm.csv')
#df_promRec.reset_index(drop=True, inplace=True)

Esta es una muestra de la información leida:

In [ ]:
df_precioProm.sample(5)

In [ ]:
df_promRec.sample(10)


# <font color='056938'> **Procesamiento de los datos** </font>

La información cargada ha sido almacenada en dos dataframes:
* `df_precioProm`: contiene los precios históricos por producto en cada plaza de mercado
* `df_promRec`: contiene las cantidades históricamente recogidas por producto en cada plaza de mercado


Crearemos dos dataframes adicionales que agrupan esta información y que nos serviran para algunos de los gráficos que construiremos

* `df_groupedprice`: contiene para cada producto su precios máximo, mínimo y promedio
* `df_groupedRec`: contiene para cada producto la cantidad total recogida en cada Departamento durante el periodo recopilado en la base de datos. De este dataframe crearemos dos versiones con dos formas de almacenar los datos

Asi se verian entonces los dos nuevos conjuntos de datos

In [ ]:
# Se usa la opción groupby para generar agrupaciones de los datos originales
df_groupedprice = df_precioProm.groupby('producto').precioPromedio.agg(
    precio_max=('max'),
    precio_min=('min'),
    precio_prom = ('mean')
)
df_groupedprice = df_groupedprice.reset_index()
df_groupedprice.head()

In [ ]:
# Se usa la opción groupby para generar agrupaciones de los datos originales
df_groupedRec = df_promRec.groupby(['artiNombre', 'deptNombre']).promedioKg.agg(
    total=('sum')
)
df_groupedRec = df_groupedRec.reset_index()
df_groupedRec.head()

Esta forma del dataframe se conoce como `long-form`, en ella se tiene una fila por observación y una columna por variable. Es común almacenar así datos multivariados con mas de dos dimensiones


Almcenemos la misma información en otro formato en un dataframe llamado `df_groupedRec_wide`

In [ ]:
df_groupedRec_wide = df_groupedRec.pivot(index='artiNombre', columns='deptNombre', values='total')
df_groupedRec_wide = df_groupedRec_wide.reset_index()
df_groupedRec_wide.head()

Esta forma del dataframe se conoce como `wide-form`. Este formato es util para guardar datos de dos dimensiones (e.g., muchas de las tablas que usualmente generamos), en el hay una fila por cada valor de la primera variable y una columna por cada valor de la segunda variable.



# <font color='056938'> **Visualización de datos** </font>
Existen diversas herramientas para generar visualizaciones de datos en python, siendo las más usadas [matplotlib](https://matplotlib.org/), [seaborn](https://seaborn.pydata.org/) y [plotly](https://plotly.com/graphing-libraries/)

En este tutorial haremos uso de plotly entre otras por las siguientes razones:
* Los gráficos son interactivos por defecto sin requerir un mayor esfuerzo de programación
* Dispone de una [documentación](https://plotly.com/python/) amplia y facilmente accesible
* Es compatible con `dash` que es una librería para la creación de los tableros web



## <font color='8EC044'> **Introducción a Plotly** </font>

Plotly en python es una libreria interactiva y de código abierto que abarca más de 40 tipos de gráficos y figuras con aplicación en estadistica, finanzas, geografia, etc.

Es importante mencionar que la libreria esta basada en el paquete generado en java script plotly.js, lo cual permite su integración en visualizaciones y tableros en aplicaciones web. Adicionalente, existen versiones de la libreria en otros lenguajes como `R`, `Julia` y `MATLAB`

Poltly tiene tres módulos principales

* `plotly.plotly` que sirve de intermediario entre el entorno local y la libreria

* `plotly.graph.objects` que contiene los objetos necesarios para crear los gráficos (i.e., Figure, layout, data, y definiciones de los graficos como scatter plot, line chart)

* `plotly.tools` que contiene herramientas implementadas como funciones para extender las posibilidades de los gráficos

Los gráficos se generán usualmente importando el segundo modulo (generalmente se usa el alias `go` para identificar la libreria importada)


```python
import plotly.graph_objects as go
```
Sin embargo, generar los gráficos a través del modulo `graph_objects` requiere un buen entendimiento de la libereria, es por ello que se ha puesto a disposición el modelo adicional `plotly.express` el que usualmente se importa con el alias de `px`

```python
import plotly.express as px
```

`Plotly Express` es un *wrapper* de `plotly.graph_objects` que simplifica la sintaxis para una gran variedad de funcionalidades de la libreria, es más simple de usar y permite crear una figura completa de forma directa. Este modulo usa internamente los objetos de `plotly.graph_objects` y retorna una instancia de la figura (`graph_objects.Figure`)

Iniciaremos entonces creando algunas Figuras con `plotly Express` y despues haremos énfasis en algunos elementos avanzados mediante `plotly.graph_objects`


## <font color='8EC044'> **Plotly Express** </font>

Comencemos por instalar la libreria y sus modulos. Note que en este caso instalamos una versión especifica de plotly pues `google colab` por defecto no siempre tienen a disposición la última versión.

In [ ]:
!pip install plotly
import plotly.express as px
import pandas as pd

###  <font color='46B8A9'> **Nuestro primer gráfico** </font>

Graficaremos el precio promedio por producto, que hemos consolidado en este dataframe

In [ ]:
df_groupedprice.head()

Con una sola linea de código podemos crear un gráfico de barras interactivo básico.

In [ ]:
fig = px.bar(df_groupedprice, x="producto", y='precio_prom')
fig.show()

Note que las eqtiquetas aparecen al posicionar el cursor sobre las barras y que es posible hacer zoom sobre secciones especificas del gráfico.

Veamos los principales elementos de la sintaxis de este gráfico:


> Usamos la libreria `px` importada  e indica que se creará un gráfico de barras.
```python
px.bar()
```

>El primer argumento `df_groupedprice` indica el dataframe que contiene los datos que usaremos.

>El segundo argumento
```python
x="producto"
```
Indica que en el eje `x` se graficará la variable `producto`. Note que esta debe ser una variable existente en el dataframe

>El tercer argumento
```python
y="precio_prom"
```
Indica que en el eje `y` se graficarán las barras correspondientes al precio promedio de cada producto




Note que es posible graficar varios barras para cada valor del eje `y` en el mismo gráfico. En este caso es necesario especificar el nombre de todas las columnas que deseamos gráficar como parte de la variable `y`

```python
y=['precio_min', 'precio_prom','precio_max']
```

Además de especificar que deseamos que sean barras separadas y no apiladas. Para ello usamos la función que actualiza el `layout` del gráfico
```python
fig.update_layout(barmode='group')
```


In [ ]:
fig = px.bar(df_groupedprice, x="producto", y=['precio_min', 'precio_prom', 'precio_max'])
fig.update_layout(barmode='group')
fig.show()

Note inclusive que no es necesario que los datos a gráficar provengan de un dataframe. En este ejemplo haremos un gráfico simple tres grupos de edad y el conteo de personas en cada grupo

In [ ]:
grupo_edad = ['menores', 'adultos', 'tercera_edad']
conteo = [10, 26, 6]
fig = px.bar(x=grupo_edad, y=conteo)
fig.show()

###  <font color='46B8A9'> **Graficar datos con datos en  wide-form** </font>

Cómo señalamos anteriormente, los datos pueden estar en `long-form` como en el caso del dataframe `df_groupedRec` o en `wide-form` como en el caso del dataframe `df_groupedRec_wide`


En primero lugar consideremos las cantidades recogidas consolidadas que se almacenaron en `long-form`en el data frame `df_groupedRec`

In [ ]:
df_groupedRec.head()

El gráfico puede generarse de la siguiente forma:

In [ ]:
fig = px.bar(df_groupedRec, x="artiNombre", y="total", color="deptNombre")
fig.show()

Note en este caso el gráfico construido con los datos en formato `long-form` en el cual las barras se apilan con diferentes colores usando la instrucción

```python
color="deptNombre"
```

Donde `deptNombre` es el nombre del departamento

El mismo gráfico puede obtenerse para los datos en el formato `wide-form` que almacenamos en el dataframe `df_groupedRec_wide`

In [ ]:
df_groupedRec_wide.head()

Para ello debemos primero obtener el nómbre de todas las columnas


```python
columns = list(df_groupedRec_wide.columns)
```

El listado de columnas es usado como parametro para las variables `y`

```python
 y=columns
```

In [ ]:
columns = list(df_groupedRec_wide.columns)
fig = px.bar(df_groupedRec_wide, x="artiNombre", y=columns)
fig.show()

## <font color='8EC044'> **Dar formato al estilo del gráfico** </font>

Hasta ahora el gráfico tiene los elementos básicos pero usualmente estamos interesados en mejorar algunos elementos de estilo como por ejemplo: adicionar titulos, ajustar los nombres de los ejes y las etiquetas. Los gráficos generados con Plotly Express son altamente estilizables. [Aquí](https://plotly.com/python/styling-plotly-express/) encontraras una descripción completa de como hacerlo

Veamos algunos ejemplos


###  <font color='46B8A9'> **Título y nombre de los ejes** </font>

En el momento de crear el gráfico podemos especificar el titulo y modificar las etiquetas de los ejes  usando las siguientes instrucciones. Donde *artiNombre*, *deptNombre* y *total* son las etiquetas actuales



```python
title="Producción total por producto en cada departamento",
labels={'artiNombre': 'producto',
        'deptNombre': 'departamento',
        'total': 'kg producidos'}
)
```




###  <font color='46B8A9'> **`Hover` o etiqueta flotante** </font>
En los gráficos interactivos automáticamente se activa una etiqueta flotante (*hover*) que se despliega cuando se pasa el cursos por la serie de datos. Por defecto esta etiqueta presenta toda la información que esta siendo gráficada, es decir, en nuestro caso *departamento*, *producto* y *cantidad*. Sin embargo, uno podría estar interesado en modificar la apariencia de dicha etiqueta

Como ejemplo, quietaremos de la etiqueta el nombre del producto puesto que ya este puede observarse en el eje x, a los kilogramos totales les daremos formato con un solo decimal y adicionaremos un dato *adicional* ficticio correspondiente a un número aleatorio con tres decimales



```python
hover_data={'artiNombre':False,
            'total':':.f',
            'adicional': (':.3f', np.random.random(len(df_groupedRec)))
            }
            
```




###  <font color='46B8A9'> **Dar formato a las etiquetas de los ejes** </font>

Existen opciones epecificas ara cambar el formato de elementos particulares del gráfico. Por ejemplo, si queremos dar formato a las etiquetas del eje y, para agregar las unidades (kg), podriamos hacerlo así:



```python
fig.update_yaxes(
    ticksuffix=" kg", showgrid=True
)
```



De momento el gráfico generado puede luce de la siguiente manera

In [ ]:
import numpy as np
import string
import random


fig = px.bar(df_groupedRec, x="artiNombre", y="total", color="deptNombre",
             title="Producción total por producto en cada departamento",
             labels={'artiNombre': 'producto',
                     'deptNombre': 'departamento',
                     'total': 'kg producidos'},
             hover_data={'artiNombre':False,
                         'total':':.f',
                         'adicional': (':.3f', np.random.random(len(df_groupedRec)))},
            )

fig.update_yaxes(
    ticksuffix=" kg", showgrid=True
)

fig.show()

###  <font color='46B8A9'> **Editar el diseño general (*update layout*)** </font>

Si los elementos creados en el gráfico por defecto y manipulables a través de sus propiedades no son suficientes, es posible [actualizar o adicionar](https://plotly.com/python/creating-and-updating-figures/) elementos una vez el gráfico ha sido creado.

Por ejemplo, el diseño general del gráfico puede editarse una vez la figura ha sido generada usando la instrucción

```python
fig.update_layout('opciones a editar')
```

Las categorias que pueden editarse de esta forma son:
* Title
* Legends
* Margins
* Size
* Fonts
* Colors

En este [enlace](https://plotly.com/python/reference/layout/) se provee una descripción detallada de las propiedades que pueden editarse en cada categoria. Adicionalmente, dependiento del tipo de traza o gráfico generado existen otra serie de propiedades que pueden editarse según sea el caso. A modo de ejemplo editaremos el color del titulo y lo centraremos, incrementaremos la altura dle gráfico y cambiaremos la fuente de todos los elementos



```python
fig.update_layout(title_x=0.5, # 0.5 corresponde a centrado
                  title_font_color='blue',
                  height=650,
                  font_family="Times New Roman")
```

Así mismo, es posible adicionar elementos una vez creado el gráfico, incluyamos por ejemplo una linea en el valor de producción de 10 millones con una nota especial para el Pimentón



```python
fig.add_shape(
    type="line", line_color="black", line_width=3, opacity=0.7, line_dash="dot",
    x0=0, x1=1, xref="paper", y0=10000000, y1=10000000, yref="y"
)

fig.add_annotation(
    text="Arriba del promedio", x="Pimentón", y=16000000, arrowhead=1, showarrow=True
)
```

De momento el gráfico sería el siguiente



In [ ]:
import numpy as np
import string
import random


fig = px.bar(df_groupedRec, x="artiNombre", y="total", color="deptNombre",
             title="Producción total por producto en cada departamento",
             labels={'artiNombre': 'producto',
                     'deptNombre': 'departamento',
                     'total': 'kg producidos'},
             hover_data={'artiNombre':False,
                         'total':':.f',
                         'adicional': (':.3f', np.random.random(len(df_groupedRec)))},
            )
fig.update_layout(title_x=0.5,
                  title_font_color='blue',
                  height=650,
                  font_family="Times New Roman")

fig.add_shape(
    type="line", line_color="black", line_width=3, opacity=0.7, line_dash="dot",
    x0=0, x1=1, xref="paper", y0=10000000, y1=10000000, yref="y"
)

fig.add_annotation(
    text="Arriba del promedio", x="Pimentón", y=16000000, arrowhead=1, showarrow=True
)

fig.show()

## <font color='8EC044'> **Otros tipos de gráficos en Plotly Express** </font>

Plotly Express provee una amplia variedad de gráficos que pueden desplegarse con muy pocas lineas de código. Un listado detallado puede consultarse en este [enlace](https://plotly.com/python/plotly-express/). A modo de ejemplo, usaremos algunos de ellos con los dataframes que creamos anteriormente


###  <font color='46B8A9'> **Gráfico de lineas** </font>
El gráfico de lineas es usualmente usado para gráficar series de tiempo. en nuestro caso, graficaremos la serie del precio de la mandarina en antioquia

In [ ]:
import plotly.express as px

# Seleccionamos los datos para la habichuela en medellin y los ordenamos segun su fecha de registro
df_habimede = df_precioProm[(df_precioProm['producto']=='Habichuela') & (df_precioProm['ciudad']=='MEDELLÍN') ]
df_habimede = df_habimede.sort_values(by=['fechaCreacion'])

# graficamos usando el tipo de grafico de linea
fig = px.line(df_habimede, x="fechaCreacion", y="precioPromedio", title='Precio promedio de la habichuela en Medelín')
fig.show()

###  <font color='46B8A9'> **Histograma** </font>

Un histograma es una representación de la distribución de un conjunto de datos numéricos, en el que estos son categoruzados en clases y se presenta el conteo para cada una de las clases.


In [ ]:
import plotly.express as px

# Seleccionamos los datos para la habichuela en medellin y los ordenamos segun su fecha de registro
df_habimede = df_precioProm[(df_precioProm['producto']=='Habichuela') & (df_precioProm['ciudad']=='MEDELLÍN') ]
df_habimede = df_habimede.sort_values(by=['fechaCreacion'])

# Se gráfica el histograma haciendo uso del tipo de grafico histogram
fig = px.histogram(df_habimede, x="precioPromedio")
fig.show()

## <font color='8EC044'> **Practica 1. Poniendo en práctica Plotly Express** </font>






###  <font color='46B8A9'> **Ejercicio 1. Gráficos de bigotes** </font>

Genere diagramas de bigotes para cada una de las ciudades, que permita visualizar la distribución de los precios de la ahuyama durante el periodo de observación. Para ello use el dataframe en que se almacenaron los precios filtrandolo `df_precioProm`  para el producto en cuestión.

In [ ]:
df_precioProm.head()
df_ahuyama = df_precioProm[df_precioProm['producto']=="Ahuyama"][['ciudad', 'precioPromedio']]
df_ahuyama.head()

In [ ]:
# Inserte aquí su respuesta


###  <font color='46B8A9'> **Ejercicio 2. Gráfico de torta** </font>
Genere un gráfico de pastel (pie chart) en el que se muestre la producción de total de Ahuyama  en los distintos Departamentos del país. Para ello use la información filtrada desde el dataframe `df_groupedRec`, asi:

In [ ]:
df_ahuyama = df_groupedRec[df_groupedRec['artiNombre']=='Ahuyama']
df_ahuyama.head()

El gráfico resultante deberá verse similar a la siguiente imagen:


![title](https://drive.google.com/uc?id=1pyx2RdE6BtI14imb_91D9YyaNSQmU5c-)



In [ ]:
# Inserte aquí su respuesta

## <font color='8EC044'> **Ploty objetos gráficos (*Graphical objects*)** </font>

Anteriormente mencionamos que `plotly.graph.objects` que contiene los objetos necesarios para crear los gráficos y que `plotly.expres` es una interfaz o wrapper que simplifica su uso.

Si bien  `plotly.expres` debería ser sufiente para la mayoria de los gráficos que podemos requerir, en algunos casos puede ser necesario tener mayor control sobre la generación del gráfico y para ello usamos `plotly.graph.objects` el que usuamente es importado como

```python
import plotly.graph_objects as go
```

La versatilidad de `plotly.graph.objects` tiene como desventaja mayor necesidad de código y conocimiento de la libreria. Los gráficos desarrollados en `plotly.expres` pueden ser desarrollados usando `plotly.graph.objects`, veamos un ejemplo



In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(name='Mínimo', x=df_groupedprice['producto'], y=df_groupedprice['precio_min']))
fig.add_trace(go.Bar(name='Promedio', x=df_groupedprice['producto'], y=df_groupedprice['precio_prom']))
fig.add_trace(go.Bar(name='Máximo', x=df_groupedprice['producto'], y=df_groupedprice['precio_max']))

fig.show()

Note que en este caso agregamos al gráfico cada una de las barras de forma anual, por ejemplo, en el caso del precio mínimo empleamos la siguiente instrucción

```
 go.Bar(name='Mínimo', x=df_groupedprice['producto'], y=df_groupedprice['precio_min']),
```



###  <font color='46B8A9'> **Ejercicio 3. Gráficos de lineas** </font>

Considere el siguiente gráfico de lineas para el precio promedio de la habichuela en Medellín


In [ ]:
df_habimede = df_precioProm[(df_precioProm['producto']=='Habichuela')  ]
df_habimede

In [ ]:
import plotly.graph_objects as go


# Seleccionamos los datos para la habichuela en MEDELLÍN y los ordenamos segun su fecha de registro
df_habimede = df_precioProm[(df_precioProm['producto']=='Habichuela') & (df_precioProm['ciudad']=='MEDELLÍN') ]
df_habimede = df_habimede.sort_values(by=['fechaCreacion'])

# Creamos el gráfico
fig = go.Figure()

# agregamos la traza al gráfico
fig.add_trace(go.Scatter(x=df_habimede["fechaCreacion"], y=df_habimede["precioPromedio"], mode='lines'))
fig.show()



Adicione a este gráfico otra linea con los datos de CALI

In [ ]:
# Inserte aquí su respuesta


Un tipo de gráficos en los que resulta particularmente útil su construcción desde el módulo `plotly.graph.objects` son los mapas. A modo de ejemplo crearemos un mapa que permita diferenciar con colors los sitios donde la comercialización de ahuyama es alta, media o baja.

Para ello, en primer lugar filtraremos el datframe de cantidades recogidas en cada central de abasto `df_promRec` para un producto y una fecha dada

In [ ]:
# Seleccionamos la cantidad de ahuyama recogida en cada plaza de mercado en la fecha indicada
selec_prod = "Ahuyama"
select_date = "2020-10-28"
df_filtered = df_promRec[(df_promRec['enmaFecha'] == select_date) & (df_promRec['artiNombre'] == selec_prod)][['fuenNombre', 'promedioKg', 'LATITUD', 'LONGITUD']]
df_filtered.reset_index(inplace=True)

Ahora calcularemos los cuantiles 25 y 75. Y crearemos una columna que indique el nivel de producción dependiendo de como se compara contra los cuantiles

| criterio        | clasificación |
|-----------------|---------------|
| menor que q25   | baja          |
| entre q25 y q75 | media         |
| mayor que q75   | alta          |

In [ ]:

# clasifica los municipios
quantiles= df_filtered['promedioKg'].quantile([0.25,0.75])
def productor(row):
    if row['promedioKg'] < quantiles.iloc[0]:
        return 'bajo'
    elif row['promedioKg'] >= quantiles.iloc[0] and row['promedioKg'] < quantiles.iloc[1]:
        return 'medio'
    else:
        return 'alto'
# apply to dataframe, use axis=1 to apply the function to every row
df_filtered['productor'] = df_filtered.apply(productor, axis=1)



El dataframe resultante sería

In [ ]:
df_filtered.head()

Ahora creamos la figura usando la libreria `plotly.graph.objects` y adicionamos cada una de las trazas. Note que en este caso hemos usado el tipo de mapa *Open Street Map* por lo que no requerimos de un token de acceso

In [ ]:
import plotly.graph_objects as go
import pandas as pd


map = go.Figure()

# adiciona ciudad con producción baja
df_bajo = df_filtered[df_filtered['productor'] =="bajo"]
map.add_trace(go.Scattermapbox(
  lat=df_bajo['LATITUD'],
  lon=df_bajo['LONGITUD'],
  mode='markers',
  marker=go.scattermapbox.Marker(
    size=12,
    color='yellow',
    opacity=0.5
  ),
  text=df_bajo['fuenNombre'],
    hoverinfo='text',
    name="prod baja"
))

# adiciona ciudad con producción media
df_medio = df_filtered[df_filtered['productor'] =="medio"]
map.add_trace(go.Scattermapbox(
  lat=df_medio['LATITUD'],
  lon=df_medio['LONGITUD'],
  mode='markers',
  marker=go.scattermapbox.Marker(
    size=14,
    color='orange',
    opacity=0.5
  ),
  text=df_medio['fuenNombre'],
    hoverinfo='text',
    name="prod media"
))

# adiciona ciudad con producción alta
df_alta = df_filtered[df_filtered['productor'] =="alto"]
map.add_trace(go.Scattermapbox(
  lat=df_alta['LATITUD'],
  lon=df_alta['LONGITUD'],
  mode='markers',
  marker=go.scattermapbox.Marker(
    size=16,
    color='red',
    opacity=0.5
  ),
  text=df_alta['fuenNombre'],
    hoverinfo='text',
    name="prod alta"
))

# Actualizamos el layout indicandole el estilo de mapa y algunas propiedades
map.update_layout(
        mapbox_style="open-street-map",
        autosize=True,
        hovermode='closest',
        showlegend=True,
        height=600
)

#Centramos el mapa y le damos el zoom apropiado
map.update_mapboxes(
  center=go.layout.mapbox.Center(
    lat=df_bajo.loc[0, 'LATITUD'],
    lon=df_bajo.loc[0, 'LONGITUD'],
  ),
  zoom=4
),
map.show()

## <font color='8EC044'> **Funciones que retornan gráficos** </font>

Para la creación de tableros es de utilidad definir funciones que dados unos argumentos construyan y retornen un gráfico

A modo de ejemplo construiremos una función que dado el conjunto de datos, el producto escogido y una fecha seleccionada, retorne un mapa como el generado anterior mente.

Para ello usaremos el concepto de funciones en python que es bastante simple:



```python
def nombre_funcion(argumento1, argumento 2, ...):
  # Cuerpo de la función

  return value
```

En nuestro caso los argumentos de entrada serían el dataframe, el producto y la fecha y el valor retornado sería el mapa


In [ ]:
import plotly.graph_objects as go
import pandas as pd

def crear_mapa(producto, fecha):

  # Filtra datos
  df_filtered = df_promRec[(df_promRec['enmaFecha'] == fecha) & (df_promRec['artiNombre'] == producto)][['fuenNombre', 'promedioKg', 'LATITUD', 'LONGITUD']]
  df_filtered.reset_index(inplace=True)

  # clasifica los municipios
  quantiles= df_filtered['promedioKg'].quantile([0.25,0.75])
  def productor(row):
      if row['promedioKg'] < quantiles.iloc[0]:
          return 'bajo'
      elif row['promedioKg'] >= quantiles.iloc[0] and row['promedioKg'] < quantiles.iloc[1]:
          return 'medio'
      else:
          return 'alto'
  # apply to dataframe, use axis=1 to apply the function to every row
  df_filtered['productor'] = df_filtered.apply(productor, axis=1)
  df_filtered.head()

  map = go.Figure()

  # adiciona ciudad con producción baja
  df_bajo = df_filtered[df_filtered['productor'] =="bajo"]
  map.add_trace(go.Scattermapbox(
    lat=df_bajo['LATITUD'],
    lon=df_bajo['LONGITUD'],
    mode='markers',
    marker=go.scattermapbox.Marker(
      size=12,
      color='yellow',
      opacity=0.5
    ),
    text=df_bajo['fuenNombre'],
      hoverinfo='text',
      name="prod baja"
  ))

  # adiciona ciudad con producción media
  df_medio = df_filtered[df_filtered['productor'] =="medio"]
  map.add_trace(go.Scattermapbox(
    lat=df_medio['LATITUD'],
    lon=df_medio['LONGITUD'],
    mode='markers',
    marker=go.scattermapbox.Marker(
      size=14,
      color='orange',
      opacity=0.5
    ),
    text=df_medio['fuenNombre'],
      hoverinfo='text',
      name="prod media"
  ))

  # adiciona ciudad con producción alta
  df_alta = df_filtered[df_filtered['productor'] =="alto"]
  map.add_trace(go.Scattermapbox(
    lat=df_alta['LATITUD'],
    lon=df_alta['LONGITUD'],
    mode='markers',
    marker=go.scattermapbox.Marker(
      size=16,
      color='red',
      opacity=0.5
    ),
    text=df_alta['fuenNombre'],
      hoverinfo='text',
      name="prod alta"
  ))

  # Actualizamos el layout indicandole el estilo de mapa y algunas propiedades
  map.update_layout(
          mapbox_style="open-street-map",
          autosize=True,
          hovermode='closest',
          showlegend=True,
          height=600
  )

  # Centramos el mapa y le damos el zoom apropiado
  map.update_mapboxes(
    center=go.layout.mapbox.Center(
      lat=df_bajo.loc[0, 'LATITUD'],
      lon=df_bajo.loc[0, 'LONGITUD'],
    ),
    zoom=4
  ),

  return map

In [ ]:
mapa = crear_mapa("Coco", "2020-10-28")
mapa.show()


###  <font color='46B8A9'> **Ejercicio 4** </font>

Genere una función que con base en el dataframe de precios (`df_precioProm`)  le permita seleccionar un producto y una ciudad y gráfique el historico de los precios.






In [ ]:
df_precioProm.head()

Para ello tenga en cuenta que dado un producto y una ciudad,  el dataframe puede filtrarse asi:

In [ ]:
producto = "Ahuyama"
ciudad = "BARRANQUILLA"
df_filtered = df_precioProm[(df_precioProm['producto']==producto) & (df_precioProm['ciudad']==ciudad)][['fechaCaptura', 'precioPromedio']]



In [ ]:
# Inserte aquí su respuesta

In [ ]:
Revisado: Daniel Felipe Tobon. 2023/07/31